In [1]:
from huggingface_hub import login
from esm.models.esm3 import ESM3
from esm.sdk.api import ESM3InferenceClient, ESMProtein, GenerationConfig

In [2]:
# Will instruct you how to get an API key from huggingface hub, make one with "Read" permission.
login()

In [4]:
# This will download the model weights and instantiate the model on your machine.
model: ESM3InferenceClient = ESM3.from_pretrained("esm3-open").to("cuda") # or "cpu"

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

data/1utn.pdb:   0%|          | 0.00/569k [00:00<?, ?B/s]

data/esm3_entry.list:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

...

esm3_function_decoder_v0.pth:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

esm3_sm_open_v1.pth:   0%|          | 0.00/2.80G [00:00<?, ?B/s]

keyword_idf_safety_filtered_58641.npy:   0%|          | 0.00/469k [00:00<?, ?B/s]

esm3_structure_decoder_v0.pth:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

esm3_structure_encoder_v0.pth:   0%|          | 0.00/62.3M [00:00<?, ?B/s]

In [18]:
# Generate a completion for a partial Carbonic Anhydrase (2vvb)

prompt = "EVQLVESGGGLVQPGGSLRLSCAASGFNIKDTYIHWVRQAPGKGLEWVARIYPTNGYTRYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYCS__G__GF_AMDYWGQGTLVTVSSGGGGSGGGGSGGGGSDIQMTQSPSSLSASVGDRVTITCRASQDVNTAVAWYQQKPGKAPKLLIYSASFLYSGVPSRFSGSRSGTDFTLTISSLQPEDFATYYCQ____TPPTFGQGTKVEIK"
prompt = "EVQLVESGGGLVQPGGSLRLSCAASGFNIKDTYIHWVRQAPGKGLEWVARIYPTNGYTRYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYCS__G__GF_AMDYWGQGTLVTVSS"

protein = ESMProtein(sequence=prompt)

In [19]:
# Generate the sequence, then the structure. This will iteratively unmask the sequence track.
protein = model.generate(protein, GenerationConfig(track="sequence", num_steps=8, temperature=0.7))

100%|██████████| 5/5 [00:00<00:00, 14.03it/s]


In [20]:
protein

ESMProtein(sequence='EVQLVESGGGLVQPGGSLRLSCAASGFNIKDTYIHWVRQAPGKGLEWVARIYPTNGYTRYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYCSRGGGSGFYAMDYWGQGTLVTVSS', secondary_structure=None, sasa=None, function_annotations=None, coordinates=None, plddt=None, ptm=None, potential_sequence_of_concern=False)

In [21]:
# We can show the predicted structure for the generated sequence.
protein = model.generate(protein, GenerationConfig(track="structure", num_steps=8))
protein.to_pdb("./generation.pdb")

100%|██████████| 8/8 [00:00<00:00, 13.70it/s]


In [23]:
# Then we can do a round trip design by inverse folding the sequence and recomputing the structure
protein.sequence = None

In [26]:
protein = model.generate(protein, GenerationConfig(track="sequence", num_steps=8))

100%|██████████| 8/8 [00:00<00:00, 14.08it/s]


In [27]:
protein

ESMProtein(sequence='EVQLEESGGGLVQPGGSLRLSCATSGFNINFTAMWWVRQAPGKGLEFVAEIYPNHGSTYYADSVKGRFTISRDNAKNIVYLQMVSLRAEDTAMYYCARQRDDSRNAMDYWGQGTQVTVSS', secondary_structure=None, sasa=None, function_annotations=None, coordinates=tensor([[[-17.2286,   5.3398,  12.3902],
         [-15.8789,   5.8861,  12.2956],
         [-14.9296,   4.8935,  11.6328],
         ...,
         [     inf,      inf,      inf],
         [     inf,      inf,      inf],
         [     inf,      inf,      inf]],

        [[-14.1443,   5.2068,  10.7684],
         [-13.2509,   4.2935,  10.0636],
         [-12.0404,   3.9385,  10.9207],
         ...,
         [     inf,      inf,      inf],
         [     inf,      inf,      inf],
         [     inf,      inf,      inf]],

        [[-11.7442,   2.6444,  10.8501],
         [-10.5726,   2.1767,  11.5834],
         [ -9.7014,   1.2783,  10.7117],
         ...,
         [     inf,      inf,      inf],
         [     inf,      inf,      inf],
         [     inf,      inf,      in

In [17]:
protein.coordinates = None
protein = model.generate(protein, GenerationConfig(track="structure", num_steps=8))
protein.to_pdb("./round_tripped.pdb")

100%|██████████| 8/8 [00:00<00:00, 13.97it/s]
